# Processing an Entire Image

Say you want to process an entire image. Pulling a full strip of imagery into a notebook is quite hefty as Maxar satellite imagery can range from 10gb to 40gb. GBDXTools offers some helpers that let us process pixels in a deferred way so that we can apply methods to work with pixels across large AOIs and only perform the computation (fetching of actual pixel data) when we need it. 

In this notebook we'll walk through a basic example of applying a band index across an entire image. We'll fetch several AOIs on the processed image and produce a large geotiff that fetch lots of data.

## Create an image 
First we want to create a CatalogImage of an entire WorldView-3 strip. Note that we don't specify a `bbox`; we want the whole thing. 

from gbdxtools import CatalogImage

image = CatalogImage('10400100137F4900')
image.shape, image.ntiles

Notice that the image is fairly large: over 2k tiles. This would be a lot to fetch all at once into memory for things like plotting.

## Define our function

For this example we'll create a simple method that accepts a chunk of imagery at a time and returns Normalize Difference Water Index (NDWI) from the Green and Near-Infrared bands (bands 2 and 7).   

In [ ]:
import numpy as np

# this method read a chunk of data
# the chunk of data can be changed in some way and returned
def ndwi(chunk):
    return np.expand_dims(np.clip((chunk[4,:,:] - chunk[7,:,:]) / (chunk[4,:,:] + chunk[7,:,:]), -1, 1), axis=0)

Our method above just calculates our water index, clips the data to -1 and 1 and returns the index. Pretty simple!

Next we use the `map_blocks` method to apply our function over every chunk of data in the image. What we have at the end is an image of NDWI data ready for use.

In [ ]:
processed = image.map_blocks(ndwi, dtype=image.dtype, chunks=(1,256,256))

In [ ]:
aoi = image.randwindow((500,500))
aoi.plot()

In [ ]:
from shapely.geometry import shape 
import matplotlib.pyplot as plt
%matplotlib inline

aoi = image.randwindow((500,500))
bounds = image.pxbounds(shape(aoi))

ndwi_aoi = processed[:, bounds[1]:bounds[3], bounds[0]:bounds[2]]

plt.axis('off')
plt.imshow(ndwi_aoi[0,:,:], cmap="jet")
plt.show()

## Check out the data

Now that we've processed our image, lets check out some random windows on the data for a visual check. Here we loop over 10 windows using `iterwindows` and plot each one. 

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(15,5))

for i, window in enumerate(processed.iterwindows(10, (256,256))):
    aoi = window.read(quiet=True)
    plt.subplot(2,5,i+1)
    plt.axis('off')
    plt.imshow(aoi[0], cmap="jet")
    
plt.show()


## Create a geotiff

Now we can use the geotiff method on our processed image to efficiently create a geotiff image of the entire strip.

In [ ]:
processed.geotiff(path="ndwi.tif")

In [ ]:
!ls -l ndwi.tif